# Eulerian and Hamiltonian Graphs

In 1736, Leonhard Euler showed that there was no way to travel across the [Seven Bridges of Königsberg](https://en.wikipedia.org/wiki/Seven_Bridges_of_K%C3%B6nigsberg), crossing each bridge a single time and ending up where you started. He did this by showing that there is no *Eulerian circuit* in a graph representing the city, where vertices represent landmasses and edges represent bridges between them. This established the first result in graph theory.

More precisely, an *Eulerian circuit* is a cycle $v_1, v_2, \ldots, v_k = v_1$ such that every edge of the graph appears as $\{v_i, v_{i+1}\}$ for some $1 \leq i \leq k$, and no edge is repeated. In other words, it is a cycle which contains every edge of the graph precisely once. You can think about this as some way of travelling around the graph, crossing each edge precisely once. Note that this does **not** mean that each vertex has to be visited precisely once (or even at all!)

We say that a graph is *Eulerian* if there is some Eulerian circuit in the graph.

Euler's realisation was that the existence of an Eulerian circuit is related to the degrees of the vertices.


***Theorem:*** a graph has an Eulerian circuit if and only if every vertex has even degree, and all of the vertices of positive degree are in one connected component.

One of the proofs of the theorem above is constructive; it defines an algorithm and then shows that the algorithm outputs an Eulerian circuit given the hypotheses of the theorem. The algorithm is as follows:

```
1. Initialise a stack of vertices: pick a vertex u in G, and set stack = [u]
2. Initialise an output circuit: set circuit = []
3. While the stack is not empty:
    4. Call the last element in the stack current_vertex.
    5. If there are any edges from current_vertex that haven't already been considered:
        6: Pick one such edge {current_vertex, neighbour}.
        7. Note that this edge cannot be used again (from either direction).
        8. Append neighbour to the stack.
    9. Otherwise, if there are no such edges, then pop current_vertex from the stack and append it to circuit.
10. Return circuit.
```
The algorithm above finds an Eulerian circuit in a *connected* graph if it has one.

-  Implement this algorithm as a function `eulerian_circuit`.

-  Describe the theoretical worst-case complexity of `eulerian_circuit`, using appropriate parameters for the input size. Is it possible for any algorithm for finding an Eulerian circuit in a connected graph to have a better worse-case complexity than this?

In [3]:
import networkx as nx
import matplotlib.pyplot as plt

# first example graph

G1 = nx.Graph([{0, 1}, {1, 2}, {2, 3}, {3, 4}, {4, 1}, {1, 3}, {0, 3}])

def eulerian_circuit(G):
    
    # initializing the vertices, stack, and circuit and setting the first vertex to be u
    
    vertices = list(G.nodes())

    u = vertices[0]
    stack = [u]
    seen_edges = []

    circuit = []
    
    # checking if all vertices have even degree i.e. if graph is Eulerian

    vertices = list(G.nodes())
    degree = [0]*len(vertices)

    for i in vertices:
        degree[i] = G.degree(i)

    even_degree = all(deg % 2 == 0 for deg in degree)
    
    if not even_degree:
        return None
    
    # if graph is Eulerian

    while stack:
        current_vertex = stack[-1]
        neighbours = G[current_vertex]
        
        # creating a list for all the edges
        
        edges = []
        
        for i in neighbours:
            edge = {current_vertex, i}
            reverse_edge = {i, current_vertex}
            
            # checking if the edge has already been seen from both directions
            
            if edge not in seen_edges or reverse_edge not in seen_edges:
                edges.append(edge)
                
            # if there are any edges remaining, pick an edge and add neighbour to the stack
            
        if len(edges) > 0:
            chosen_edge = edges[0]
            seen_edges.append(chosen_edge)
            neighbour = (chosen_edge - {current_vertex}).pop()
            stack.append(neighbour)
            
            # if no edges left, add the current edge to the circuit
            
        else:
            a = stack.pop()
            circuit.append(a)
    
    if circuit == []:
        return None
    return circuit

eulerian_circuit(G1)

[0, 3, 1, 4, 3, 2, 1, 0]

*Answer:*

*The theoretical worst-case complexity of `eulerian_circuit` is O(E), where E is the number of edges. This is because the time taken to find the Eulerian circuit is proportional to the number of edges in the graph, but the traversal time for each edge is independent of the number of edges. The function traverses each edge when checking if it has already been seen and once while adding to the `seen_edges` list. Thus, in the worst case scenario, each edge will be considered twice.*

*It is not possible for any algorithm to find a Eulerian circuit in a connected graph to have a better worst case complexity than this. Finding a Eulerian circuit requires each edge to be considered at least once. Thus, any such function would have at least O(E) worst-case time complexity.*

In order to apply the theorem, we need to be able to check whether all of the positive-degree vertices are connected. One way of doing this is to start a traversal from any positive-degree vertex, and check if it visits all of the others.


- Implement a function `all_positive_degree_vertices_connected` which takes a `networkx` `Graph` and returns `True` if all vertices of positive degree are connected, and `False` otherwise. This should use a depth-first or breadth-first traversal.

- Describe the complexity of deciding whether a graph is Eulerian using:
    1. A combination of the functions `eulerian_circuit` and `all_positive_degree_vertices_connected` (i.e. testing whether `eulerian_circuit` returns `None` or `all_positive_degree_vertices_connected` returns `False`).    
    2. Directly applying the characterisation using degrees as described in the introduction.
    - In other words, answer the question "*can we achieve a better theoretical complexity using the theorem above if we only want to **decide** whether there is an Eulerian circuit, rather than find one*?"



In [2]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import deque

# second example graph

G2 = nx.Graph([{0, 1}, {1, 2}, {2, 3}, {3, 4}, {4, 1}, {1, 3}, {0, 3}])
G2.add_nodes_from(range(5, 8))

def all_positive_degree_vertices_connected(G):
    
    # initializing to perform breadth first traversal
    
    seen = set()
    queue = deque()
    vertices = list(G.nodes())
    
    #obtaining a list of the positive degree vertices
    
    positive_degree = vertices
    for i in positive_degree:
        if G.degree(i) <= 0:
            positive_degree.remove(i)
            
    # initializing the queue
    
    u = vertices[0]
    queue.append(u)
    
    while queue:
        current_vertex = queue.popleft()
        
        # checking if the vertex has been seen before
        
        if current_vertex not in seen:
            seen.add(current_vertex)            
            neighbours = G[current_vertex]
            
            # marking all neighbours as seen
            
            for i in neighbours:
                if i not in seen and i not in queue:
                    queue.append(i)
                    
    # checking if all positive degree vertices have been seen
                    
    if set(positive_degree)==set(seen):
        return True
        
    return False

all_positive_degree_vertices_connected(G1), all_positive_degree_vertices_connected(G2)

(True, False)

*Answer:*

*The `all_positive_degree_vertices_connected` function uses a breadth first traversal starting from one vertex to check whether all vertices with positive degree are connected. In the worst case, each vertex and each edge is visited once, and each vertex is visited once again while checking whether all positive degree vertices have been seen. Thus, it has overall time complexity O(V+E), where V is the number of vertices and E is the number of edges in the graph.*

*1. A combination of the two functions `eulerian_circuit` and `all_positive_degree_vertices_connected` would have a worst case time complexity of O(E)+O(V+E), which is simply O(V+E). This is when both functions need to be considered and are implemented sequentially.*


*2. Only deciding whether a graph is Eulerian or not using the fact that Eulerian graphs have vertices with even degree has time complexity O(V). This is because it considered every vertex once while calculating its degree, and once more while deciding whether the degree is even. Thus, simply deciding if a graph is Eulerian has better time complexity than finding a Eulerian circuit.*   

In the same way that an *Eulerian* cycle is one which contains every edge precisely once, a *Hamiltonian* cycle is one which contains every *vertex* precisely once (apart from whichever vertex is picked to be "first", which must be visited twice to obtain a cycle). Determinining whether a graph is *Hamiltonian* (contains a Hamiltonian cycle) is significantly harder than determining whether it is Eulerian. In particular, it is NP-complete.


- Implement a function `hamiltonian_cycle` which uses a backtrack search to find a Hamiltonian cycle in a graph, if one exists.
- Use your function to find a Hamiltonian cycle in the Hoffman-Singleton graph.

Details:
1. Your function should return `None` if the graph is not Hamiltonian.
2. You should return a `list` of vertices representing the cycle; the first vertex in the list should also appear at the end to close the cycle.

In [4]:
import networkx as nx
import matplotlib.pyplot as plt

def consistency_check(G, path):
    vertices = list(G.nodes())
    n = len(vertices) 
    
    # checks that there are no repeated vertices in the path
    
    if len(set(path)) == len(path):
        return True
    return False


def hamiltonian_cycle(G):
    
    # initializing the path
    
    vertices = list(G.nodes())
    n = len(vertices) 
    path = []
    
    # creating the backtracking inner function

    def dive():
        
    # checking if the path is a full Hamiltonian cycle: if every vertex appears once and length of path is equal to number of vertices
    
        if len(path) == n:
            if consistency_check(G, path):
                return path
            return None
            
        # modifying the path using adjacent vertices that have not been seen before
        
        neighbours = G[path[-1]]
        
        for i in neighbours:
            if G.has_edge(i, path[-1]):
                if i not in path:
                
                # if the solution is consistent, returning dive() and if not, removing this vertex from the path 
                
                    path.append(i)
                    if consistency_check(G, path):
                        res = dive()
                        if res is not None:
                            return res
                    path.pop
                    
        return None
    
    # trying a path from every vertex
    
    for v in vertices:
        path = [v]
        res = dive()
        
        # completing the cycle if the path is Hamiltonian
        
        if res is not None and G.has_edge(res[0],res[-1]):
            
            res.append(res[0])
            return res
        
    return None        

# checking the function with the Hoffman Singleton graph

G3 = nx.hoffman_singleton_graph()

res = hamiltonian_cycle(G3)

res

# Thus, the Hoffman Singleton graph is Hamiltonian and one Hamiltonian cycle is given by 'res'.

[5,
 3,
 0,
 1,
 17,
 4,
 11,
 2,
 10,
 18,
 6,
 22,
 13,
 30,
 19,
 7,
 34,
 15,
 35,
 25,
 9,
 32,
 23,
 14,
 31,
 12,
 36,
 8,
 33,
 16,
 39,
 28,
 20,
 37,
 29,
 21,
 40,
 41,
 24,
 38,
 26,
 42,
 43,
 44,
 27,
 47,
 45,
 46,
 49,
 48,
 5]

In [5]:
# creating a function to see if the graph is connected

import networkx as nx
import matplotlib.pyplot as plt
from collections import deque

def all_vertices_connected(G):
    
    #initializing to perform breadth first traversal
    
    seen = set()
    queue = deque()
    vertices = list(G.nodes())
   
    u = vertices[0]
    queue.append(u)
    
    while queue:
        current_vertex = queue.popleft()
        
        #checking if the vertex has been seen before
        
        if current_vertex not in seen:
            seen.add(current_vertex)            
            neighbours = G[current_vertex]
            
            #marking all neighbours as seen
            
            for i in neighbours:
                if i not in seen and i not in queue:
                    queue.append(i)
                    
    #checking if all vertices have been seen
    
    if set(vertices)==set(seen):
        return True
    return False

# creating a function to check if a graph is eulerian
    
def is_eulerian(G):
    
    # using the functions described above, easy to check if the graph has a Eulerian circuit
    
    # all vertices should be connected
    
    if not all_vertices_connected(G):
        return False
    
    vertices = list(G.nodes())
    n = len(vertices)
    degree = [0]*len(vertices)

    for i in vertices:
        degree[i] = G.degree(i)

    even_degree = all(deg % 2 == 0 for deg in degree)
    
    if not even_degree:
        return False
    return True
    
def eulerian_hamiltonian(G):
    
    vertices = list(G.nodes())
    n = len(vertices)
    degree = [0]*len(vertices)

    for i in vertices:
        degree[i] = G.degree(i)
    
    if not is_eulerian(G):
        return False
    
    # cannot be Hamiltonian if vertices are less than 3
    
    if n < 3:
        return False
    
    # Dirac's theorem
    
    a = all(deg >= n/2 for deg in degree)
        
    # Ore's theorem
    
    for i in vertices:
        for j in vertices:
            if i != j and not G.has_edge(i, j):
                if degree[i]+degree[j] < n:
                    b = False
            else:
                b = True
                
    # Returning False if neither of these conditions are satisfied
    
    if not a and not b:
        return False
    
    return ("The graph definitely has both a Eulerian circuit and a Hamiltonian Cycle")
    
G4 = nx.complete_graph(9)
T4 = nx.from_graph6_bytes(b"Y????????????????????????????w?F?o??o??GW?@?W?A?B?Q?K?`?")

all_vertices_connected(G4), is_eulerian(G4), eulerian_hamiltonian(G4), all_vertices_connected(T4), is_eulerian(T4), eulerian_hamiltonian(T4)


(True,
 True,
 'The graph definitely has both a Eulerian circuit and a Hamiltonian Cycle',
 True,
 False,
 False)

*A Eulerian circuit is a Eulerian path that starts and ends at the same vertex. A graph has a Eulerian circuit if and only if the graph is connected and all vertices in the graph have even degree. A graph does not contain a Eulerian or Hamiltonian circuit if it has less than three vertices.*

*Dirac's theorem states that an n-vertex graph in which each vertex has degree at least n/2 must have a Hamiltonian cycle. Ore's theorem states that if a graph has n ≥ 3 vertices, and for each pair of non-adjacent vertices u and v the sum of degrees of u and v ≥ n, then the graph is Hamiltonian.*

*The function `all_vertices_connected` first checks if the graph is connected or not and is a modified version of the function from part 2. The function `is_eulerian` first checks if a graph is connected and then checks if it is Eulerian using the conditions specified above.*

*The function `eulerian_hamiltonian` first verifies that a graph is connected and Eulerian. It then uses Dirac's theorem and Ore's theorem and returns `False` if neither of these are satisfied. Thus, if a graph passes all the conditions, it is definitely Eulerian and Hamiltonian. However, if the function returns false, the graph might still be Hamiltonian (assuming it is Eulerian).*

*From the code, we can see that a cycle graph is Eulerian and Hamiltonian, but a (connected) tree graph is not eulerian and likely not Hamiltonian.*